In [1]:
import sys
import os

sys.path.append("../Pre_processing")

from Data_Preparation_Library import *
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

image_size=128

Using TensorFlow backend.


In [6]:
#selectedBatches=["6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]
maxBatchId = 1
selectedBatches=[str(i) for i in range(maxBatchId)]

extra_folder="128_128"
batch_data_object = []
for i in selectedBatches:
    with open(os.path.join(temp_path,extra_folder,'full_data_object_' + i + '.p'), 'rb') as handle:
        batch_data_object+=pickle.load(handle)

data_train = batch_data_object[0:int(len(batch_data_object)*2/3)]
data_test  = batch_data_object[int(len(batch_data_object)*2/3):]
data_test  = batch_data_object[int(len(batch_data_object)*2/3):]

In [7]:
def flux(d):
  # TODO: get flux_i from model
    if d.i_image != None:
        flux_i = sum(sum(d.i_image))
    else:
        flux_i = 0
    flux_g = sum(sum(d.g_image))
    #return flux_i+flux_g, flux_g-flux_i
    return flux_g

In [10]:
with open(os.path.join(os.path.join(output_path,"results_128_on_128"),"encoder_results_train_v2_0"),'rb') as handle:
    encoded_imgs1=pickle.load(handle)
    encoded_imgs_reshaped_train = [e.reshape(-1,1) for e in encoded_imgs1]

with open(os.path.join(os.path.join(output_path,"results_128_on_128"),"encoder_results_test_v2_0"),'rb') as handle:
    encoded_imgs2=pickle.load(handle)
    encoded_imgs_reshaped_test = [e.reshape(-1,1) for e in encoded_imgs2]


In [11]:
def pca_feature(d):
    
#  return [v1_i,v2_i,v1_i/v2_i,v2_i/v1_i, v1_g,v2_g,v1_g/v2_g,v2_g/v1_g]


SyntaxError: unexpected EOF while parsing (<ipython-input-11-f89bb59a94fa>, line 3)

In [12]:
# accessible in the class: size of image, maximum of image, normalization factor of g + fitted value of normalization factor for i
# define a function to get the training and testing dataset

def obtain_train_data():
    train_features = []
    train_labels = []
    for i in range(len(data_train)):
        if data_train[i].logMstar!=-99:
            e = encoded_imgs_reshaped_train[i]
            d = data_train[i]
            #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            img_max=d.g_image.max()
            train_features.append(np.append(e,np.array([f,size_1,size_2,img_max])))
            train_labels.append(d.logMstar)
    return train_features, train_labels

train_features = np.asarray(obtain_train_data()[0])
train_labels = np.asarray(obtain_train_data()[1])

print(train_features.shape, train_labels.shape)

(659, 4100) (659,)


In [13]:
def obtain_test_data():
    test_features = []
    test_labels = []
    for j in range(len(data_test)):
        if data_test[j].logMstar!=-99:
            e = encoded_imgs_reshaped_test[j]
            d = data_test[j]

        #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            img_max=d.g_image.max()
            test_features.append(np.append(e,np.array([f,size_1,size_2,img_max])))
            test_labels.append(d.logMstar)

    return test_features, test_labels

test_features = np.asarray(obtain_test_data()[0])
test_labels = np.asarray(obtain_test_data()[1])

print(test_features.shape, test_labels.shape)

(325, 4100) (325,)


In [14]:
# ## transform features
# train_features = StandardScaler().fit_transform(train_features)
# test_features = StandardScaler().transform(test_features)
# #features = MinMaxScaler().fit_transform(features)

X_scaler = StandardScaler()
train_features = X_scaler.fit_transform(train_features)
test_features = X_scaler.transform(test_features)

print(train_features[:5], test_features[:5])

[[-0.46823421 -0.5080603   0.02156935 ..., -0.08512842 -0.08512842
  -0.31407447]
 [-0.37778186  0.01002213  0.36101057 ...,  1.97956789  1.97956789
  -0.3734218 ]
 [-0.31466022 -0.04653447  0.02059651 ...,  1.35231838  1.35231838
   1.8731427 ]
 [-0.34544549  0.51801735  0.59461292 ..., -0.45102397 -0.45102397
  -0.4427597 ]
 [ 0.5452832   1.31677986  0.74994534 ..., -0.56863325 -0.56863325
  -0.42390013]] [[ 0.15133955 -1.55998065 -0.05018915 ...,  0.01941317  0.01941317
  -0.41906929]
 [-0.31818065  0.05129635  0.23653529 ...,  1.888094    1.888094
  -0.11246488]
 [ 0.8351674   1.77883233  1.02854044 ..., -0.49022706 -0.49022706
  -0.44070063]
 [-0.48722797 -0.36853051  0.03476775 ...,  0.01941317  0.01941317
  -0.30787221]
 [-0.51023902 -1.52314199 -1.47030972 ..., -0.52943016 -0.52943016
  -0.42655866]]


In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Dropout

In [16]:
model = Sequential()
model.add(Dense(64, activation='sigmoid', input_dim=train_features.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(optimizer='Adadelta',
              loss='mse',
              metrics=['mse'])

In [17]:
model.fit(train_features, train_labels, epochs=100, batch_size=32,shuffle=True,
                validation_data=(test_features, test_labels))

Train on 659 samples, validate on 325 samples
Epoch 1/100
659/659 [==============================] - 1s - loss: 82.9382 - mean_squared_error: 82.9382 - val_loss: 76.9822 - val_mean_squared_error: 76.9822
Epoch 2/100
659/659 [==============================] - 0s - loss: 69.6037 - mean_squared_error: 69.6037 - val_loss: 66.1624 - val_mean_squared_error: 66.1624
Epoch 3/100
659/659 [==============================] - 0s - loss: 59.9456 - mean_squared_error: 59.9456 - val_loss: 57.2448 - val_mean_squared_error: 57.2448
Epoch 4/100
659/659 [==============================] - 0s - loss: 52.1291 - mean_squared_error: 52.1291 - val_loss: 49.9612 - val_mean_squared_error: 49.9612
Epoch 5/100
659/659 [==============================] - 0s - loss: 45.0333 - mean_squared_error: 45.0333 - val_loss: 43.7040 - val_mean_squared_error: 43.7040
Epoch 6/100
659/659 [==============================] - 0s - loss: 39.3512 - mean_squared_error: 39.3512 - val_loss: 38.3976 - val_mean_squared_error: 38.3976
Epoch 

659/659 [==============================] - 0s - loss: 0.7607 - mean_squared_error: 0.7607 - val_loss: 0.4845 - val_mean_squared_error: 0.4845
Epoch 54/100
659/659 [==============================] - 0s - loss: 0.6915 - mean_squared_error: 0.6915 - val_loss: 0.4927 - val_mean_squared_error: 0.4927
Epoch 55/100
659/659 [==============================] - 0s - loss: 0.7722 - mean_squared_error: 0.7722 - val_loss: 0.4753 - val_mean_squared_error: 0.4753
Epoch 56/100
659/659 [==============================] - 0s - loss: 0.7903 - mean_squared_error: 0.7903 - val_loss: 0.4838 - val_mean_squared_error: 0.4838
Epoch 57/100
659/659 [==============================] - 0s - loss: 0.7805 - mean_squared_error: 0.7805 - val_loss: 0.4782 - val_mean_squared_error: 0.4782
Epoch 58/100
659/659 [==============================] - 0s - loss: 0.7628 - mean_squared_error: 0.7628 - val_loss: 0.4607 - val_mean_squared_error: 0.4607
Epoch 59/100
659/659 [==============================] - 0s - loss: 0.7468 - mean_sq

In [18]:
type(test_predict),type(test_labels[1])

NameError: name 'test_predict' is not defined

In [19]:
from sklearn.metrics import mean_squared_error

test_predict = []
test_labels_2=[]
for i in range(100):
    if data_test[i].logMstar!=-99:
        test_predict.append(model.predict(test_features[i].reshape(1,-1))[0][0])
        test_labels_2.append(test_labels[i])
print(len(test_predict))
print(np.sqrt(mean_squared_error(np.asarray(test_predict), np.asarray(test_labels_2))))


100
0.500847115178


In [20]:
test_predict

[10.39009,
 10.596516,
 9.9460144,
 10.875796,
 10.110703,
 10.915573,
 10.301353,
 10.591532,
 10.894402,
 10.720873,
 10.899636,
 9.1435261,
 10.378956,
 10.841615,
 10.126148,
 10.12937,
 9.6394634,
 10.31321,
 10.862532,
 10.886653,
 10.855471,
 10.354572,
 10.186045,
 10.895294,
 10.600686,
 10.567462,
 10.682447,
 10.368379,
 10.640563,
 10.776237,
 10.888296,
 9.9669533,
 10.897311,
 10.877956,
 10.827045,
 10.79487,
 10.133039,
 10.750184,
 10.898733,
 10.206895,
 10.529582,
 10.786904,
 10.894168,
 9.9745655,
 10.766259,
 10.832606,
 10.703841,
 9.8225679,
 10.138071,
 10.70086,
 10.219549,
 10.151847,
 10.884804,
 10.104679,
 10.856207,
 10.771027,
 10.190756,
 10.825233,
 10.113098,
 10.783353,
 10.853107,
 10.671546,
 10.835136,
 10.299541,
 10.927578,
 10.904915,
 10.892394,
 10.851261,
 10.973289,
 9.4683981,
 10.814553,
 10.770698,
 9.712779,
 10.352628,
 9.7199116,
 10.569611,
 10.654953,
 10.955105,
 10.611215,
 10.915396,
 10.728649,
 10.986729,
 10.283662,
 10.777243